## Aim: combine my work with solution

There are some parts of the Benson Solutions I like, and some parts where I like my approach better. I'm going to attempt to blend the two here

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


import sys
sys.path.append('./code')
import get_mta_data as gmd
import process_mta_data as pmd

# import datetime
from datetime import datetime as dt

## import data

In [2]:
mta = gmd.load_local_data([2019], [4])

## Tidy up data

The functions that are becoming standard for us.

In [3]:
mta = pmd.clean_col_names(mta)

In [4]:
mta = pmd.add_datetime(mta)

In [5]:
mta = pmd.convert_date_to_datetime(mta)



In [6]:
mta

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,2019-03-30 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,04:00:00,REGULAR,6999084,2373576,2019-03-30 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,08:00:00,REGULAR,6999107,2373622,2019-03-30 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,12:00:00,REGULAR,6999214,2373710,2019-03-30 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,16:00:00,REGULAR,6999451,2373781,2019-03-30 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
751231,R161B,R452,00-03-02,72 ST,123,IRT,2019-04-25,01:00:00,REGULAR,20605432,8466842,2019-04-25 01:00:00
751232,R161B,R452,00-03-02,72 ST,123,IRT,2019-04-25,05:00:00,REGULAR,20605443,8466850,2019-04-25 05:00:00
751233,R161B,R452,00-03-02,72 ST,123,IRT,2019-04-25,09:00:00,REGULAR,20606268,8467023,2019-04-25 09:00:00
751234,R161B,R452,00-03-02,72 ST,123,IRT,2019-04-25,13:00:00,REGULAR,20607206,8467308,2019-04-25 13:00:00


For convenience, add a 'turnstile_id' column

In [7]:
mta['turnstile_id'] = mta.CA + mta.UNIT + mta.SCP

Find and remove duplicate values

In [8]:
mta.DESC.value_counts()

REGULAR       747615
RECOVR AUD      3621
Name: DESC, dtype: int64

There are 3621 rows with 'RECOVER AUD'. In Lara's presentation, these were shown to be be duplicates in at least some cases. We will remove them all.

In [9]:
mta = mta[mta.DESC != 'RECOVR AUD']
mta.DESC.value_counts()

REGULAR    747615
Name: DESC, dtype: int64

This confirms that the 'RECOVER AUD' entries have been removed

Here's what the data looks like now:

In [10]:
mta.head()

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,turnstile_id
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,2019-03-30 00:00:00,A002R05102-00-00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,04:00:00,REGULAR,6999084,2373576,2019-03-30 04:00:00,A002R05102-00-00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,08:00:00,REGULAR,6999107,2373622,2019-03-30 08:00:00,A002R05102-00-00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,12:00:00,REGULAR,6999214,2373710,2019-03-30 12:00:00,A002R05102-00-00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,16:00:00,REGULAR,6999451,2373781,2019-03-30 16:00:00,A002R05102-00-00


Let's check DATETIME for value counts

In [11]:
mta.DATETIME.dt.time.value_counts()

16:00:00    63450
04:00:00    63418
00:00:00    63394
20:00:00    63370
08:00:00    63331
            ...  
20:49:52        1
15:50:02        1
10:37:34        1
05:17:17        1
09:02:13        1
Name: DATETIME, Length: 30247, dtype: int64

This shows that there are some entries at odd times. There appear to be more at standardized times.  
So, let's isolate those at 00:00:00 time

In [12]:
mask = mta['DATETIME'].dt.time == dt(2016, 1, 1, 0, 0, 0).time() # date is arbitrary, important part is time
mta = mta[mask]
mta.DATETIME.dt.time.value_counts()

00:00:00    63394
Name: DATETIME, dtype: int64

This confirms that all entries are now at midnight

Now, check that "CA", "UNIT", "SCP", "STATION", "DATETIME" is unique


In [13]:
mta_groups_1 = (mta
 .groupby(["CA", "UNIT", "SCP", "STATION", "DATETIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False))
mta_groups_1.ENTRIES

0        1
42252    1
42254    1
42255    1
42256    1
        ..
21136    1
21137    1
21138    1
21139    1
63393    1
Name: ENTRIES, Length: 63394, dtype: int64

Okay, that shows that the rows are all unique in terms of "CA", "UNIT", "SCP", "STATION", and "DATETIME".

Let's check that turnstile_id groupby gives the same result

In [14]:
mta_groups_2 = (mta
    .groupby(['turnstile_id', 'DATETIME'])
    ['ENTRIES'].count()
    .reset_index()
    .sort_values("ENTRIES", ascending=False))
mta_groups_2.ENTRIES

0        1
42252    1
42254    1
42255    1
42256    1
        ..
21136    1
21137    1
21138    1
21139    1
63393    1
Name: ENTRIES, Length: 63394, dtype: int64

In [15]:
assert (mta_groups_2.ENTRIES == mta_groups_1.ENTRIES).all()

The assertion passed, confirming that those two are the same, so I can use them equivalently

In [16]:
turnstiles_daily = (mta.groupby(["CA", "UNIT", "SCP", "STATION", "DATE"])
                    ['ENTRIES', 'DATETIME', 'turnstile_id'].first()).reset_index()

In [17]:
turnstiles_daily

,CA,UNIT,SCP,STATION,DATE,ENTRIES,DATETIME,turnstile_id
0,A002,R051,02-00-00,59 ST,2019-03-30,6999064,2019-03-30,A002R05102-00-00
1,A002,R051,02-00-00,59 ST,2019-03-31,6999957,2019-03-31,A002R05102-00-00
2,A002,R051,02-00-00,59 ST,2019-04-01,7000528,2019-04-01,A002R05102-00-00
3,A002,R051,02-00-00,59 ST,2019-04-02,7002087,2019-04-02,A002R05102-00-00
4,A002,R051,02-00-00,59 ST,2019-04-03,7003680,2019-04-03,A002R05102-00-00
...,...,...,...,...,...,...,...,...
63389,S101A,R070,01-05-01,ST. GEORGE,2019-04-15,625,2019-04-15,S101AR07001-05-01
63390,S101A,R070,01-05-01,ST. GEORGE,2019-04-16,626,2019-04-16,S101AR07001-05-01
63391,S101A,R070,01-05-01,ST. GEORGE,2019-04-17,626,2019-04-17,S101AR07001-05-01
63392,S101A,R070,01-05-01,ST. GEORGE,2019-04-18,626,2019-04-18,S101AR07001-05-01


Filter down to monday midnight through saturday midnight

In [18]:
turnstiles_daily.DATETIME.dt.dayofweek

0        5
1        6
2        0
3        1
4        2
        ..
63389    0
63390    1
63391    2
63392    3
63393    4
Name: DATETIME, Length: 63394, dtype: int64

In [20]:
turnstiles_daily = turnstiles_daily[((turnstiles_daily.DATETIME.dt.dayofweek == 0) 
 | (turnstiles_daily.DATETIME.dt.dayofweek == 1)
 | (turnstiles_daily.DATETIME.dt.dayofweek == 2)
 | (turnstiles_daily.DATETIME.dt.dayofweek == 3)
 | (turnstiles_daily.DATETIME.dt.dayofweek == 4))]

In [21]:
turnstiles_daily

,CA,UNIT,SCP,STATION,DATE,ENTRIES,DATETIME,turnstile_id
2,A002,R051,02-00-00,59 ST,2019-04-01,7000528,2019-04-01,A002R05102-00-00
3,A002,R051,02-00-00,59 ST,2019-04-02,7002087,2019-04-02,A002R05102-00-00
4,A002,R051,02-00-00,59 ST,2019-04-03,7003680,2019-04-03,A002R05102-00-00
5,A002,R051,02-00-00,59 ST,2019-04-04,7005332,2019-04-04,A002R05102-00-00
6,A002,R051,02-00-00,59 ST,2019-04-05,7006970,2019-04-05,A002R05102-00-00
...,...,...,...,...,...,...,...,...
63389,S101A,R070,01-05-01,ST. GEORGE,2019-04-15,625,2019-04-15,S101AR07001-05-01
63390,S101A,R070,01-05-01,ST. GEORGE,2019-04-16,626,2019-04-16,S101AR07001-05-01
63391,S101A,R070,01-05-01,ST. GEORGE,2019-04-17,626,2019-04-17,S101AR07001-05-01
63392,S101A,R070,01-05-01,ST. GEORGE,2019-04-18,626,2019-04-18,S101AR07001-05-01


In [22]:
# turnstiles_daily.loc[:,'turnstile_id']

In [23]:
# turnstiles_daily.loc[:,"SCP"]

Add columns for previous date and previous entries

In [24]:
# turnstiles_daily.loc[:,["PREV_DATE", "PREV_ENTRIES", 'DATETIME']]

In [31]:
turnstiles_daily.loc[:,["PREV_DATE", "PREV_ENTRIES", 'DATETIME']]

,PREV_DATE,PREV_ENTRIES,DATETIME
2,NaN,NaN,2019-04-01
3,NaN,NaN,2019-04-02
4,NaN,NaN,2019-04-03
5,NaN,NaN,2019-04-04
6,NaN,NaN,2019-04-05
...,...,...,...
63389,NaN,NaN,2019-04-15
63390,NaN,NaN,2019-04-16
63391,NaN,NaN,2019-04-17
63392,NaN,NaN,2019-04-18


In [32]:
 (turnstiles_daily
                                                       .groupby(["CA", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES", 'DATETIME']
                                                       .transform(lambda grp: grp.shift(1)))


,DATE,ENTRIES,DATETIME
2,NaT,NaN,NaT
3,2019-04-01,7000528.0,2019-04-01
4,2019-04-02,7002087.0,2019-04-02
5,2019-04-03,7003680.0,2019-04-03
6,2019-04-04,7005332.0,2019-04-04
...,...,...,...
63389,2019-04-12,625.0,2019-04-12
63390,2019-04-15,625.0,2019-04-15
63391,2019-04-16,626.0,2019-04-16
63392,2019-04-17,626.0,2019-04-17


In [26]:
turnstiles_daily.loc[:,["PREV_DATE", "PREV_ENTRIES", 'DATETIME']] = (turnstiles_daily
                                                       .groupby(["CA", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES", 'DATETIME']
                                                       .transform(lambda grp: grp.shift(1)))


KeyError: "Columns not found: 'PREV_ENTRIES', 'PREV_DATE'"

And add a today - prev column

In [ ]:
turnstiles_daily.loc['day_minus_prev_entries'] = (turnstiles_daily.ENTRIES - turnstiles_daily.PREV_ENTRIES)

In [ ]:
turnstiles_daily.head()

In [ ]:
turnstiles_daily.tail()

Drop the first date rows that have NaN entries

In [ ]:
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [ ]:
turnstiles_daily.head()

Sanity check that today's entries are greater than tomorrow's entries


In [ ]:
try:
    assert (turnstiles_daily["ENTRIES"] > turnstiles_daily["PREV_ENTRIES"]).all()
except:
    print("assertion failed")

Let's look at the rows where yesterday's counts are greater than today's counts

In [ ]:
yesterday_greater = turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]]

yesterday_greater.head()

In [ ]:
yesterday_greater.groupby(['turnstile_id']).count().head(10)

Many of these have 27 entries. Let's check out one

In [ ]:
turnstiles_daily[turnstiles_daily.turnstile_id == 'C021R21200-00-02'].sort_values('DATETIME').head(10)

In [ ]:
turnstiles_daily.count()

In [ ]:
turnstiles_daily[turnstiles_daily.day_minus_prev_entries < 0].count()

What percentage of data do we lose if we through out the negative values?

In [ ]:
752 / 60909

We would lose 1.2%. I'm okay with that.
Eliminate negative values of day_minus_prev_entries

In [ ]:
turnstiles_daily = turnstiles_daily[turnstiles_daily.day_minus_prev_entries > 0]
turnstiles_daily.head()

Looking at this data, the day_minus_prev_entries column tells us how many peole entered on a given day at a given turnstile.
Now, I'd like to group by station and sum over date to get total entries

In [ ]:
turnstiles_daily.groupby('STATION').day_minus_prev_entries.sum().sort_values(ascending=False).head(20)